In [14]:
import gymnasium as gym
env = gym.make('InvertedPendulum-v4')

# 動作確認

In [10]:
import gymnasium as gym
import numpy as np
from tqdm import trange

env = gym.make('CartPole-v1', render_mode="human")

max_number_of_steps = 1000
num_consecutive_iterations = 100
num_episodes = 300
last_time_steps = np.zeros(num_consecutive_iterations)

for episode in trange(num_episodes):
    # 環境の初期化
    observation = env.reset()
    episode_reward = 0

    for t in range(max_number_of_steps):
        # action = [np.random.choice([-3, 3])]
        action = 1

        # 行動の実行とフィードバックの取得
        observation, reward, done, _, info = env.step(action)
        episode_reward += reward

        if done:
            # print('%d Episode finished after %d time steps / mean %f' % (episode, t + 1,
            #     last_time_steps.mean()))
            last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break

env.close()

100%|██████████| 300/300 [01:03<00:00,  4.71it/s]


# REINFORCE

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import trange
import gymnasium as gym
import matplotlib.pyplot as plt

NUM_DIZITIZED = 20 # 状態の分割数

#記録用
record_episode = []
record_reward = []
record_step = []
record_episode_log_probs = []
record_loss = []

# ポリシーネットワークの定義
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 24)
        # self.fc2 = nn.Linear(24, 24)
        # self.fc3 = nn.Linear(24, 24)
        # self.fc4 = nn.Linear(24, 24)
        # self.fc5 = nn.Linear(24, 24)
        # self.fc6 = nn.Linear(24, 24)
        self.fc_end = nn.Linear(24, 2)  # アクションの数は2（左右）

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        # x = F.relu(self.fc6(x))
        x = self.fc_end(x)
        return F.softmax(x, dim=1)

# ポリシーネットワークの初期化
policy_network = PolicyNetwork()

# 最適化アルゴリズムの設定
optimizer = optim.Adam(policy_network.parameters(), lr=1)

# 行動の選択関数
def select_action(state):
    # 状態をテンソルに変換
    state = torch.from_numpy(state).float().unsqueeze(0)
    # ポリシーネットワークによるアクションの確率の取得
    action_probs = policy_network(state)
    # カテゴリカル分布からアクションをサンプリング
    action_distribution = torch.distributions.Categorical(action_probs)
    action = action_distribution.sample()
    # アクションとその対数確率を返す
    return action.item(), action_distribution.log_prob(action)

# ポリシーネットワークの更新関数
def update_policy(episode_rewards, episode_log_probs, gamma=0.999):
    discounted_rewards = []
    running_add = 0
    exp_num = 0

    # 割引報酬の計算
    for r in reversed(episode_rewards):
        running_add = running_add * gamma + r
        discounted_rewards.insert(0, running_add)
        exp_num = exp_num + 1

    # リストからTensorに変換
    discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32, requires_grad=True) #デフォルトでは，requires_grad=Falseであるので注意
    episode_log_probs = torch.tensor(episode_log_probs, dtype=torch.float32, requires_grad=True)


    # ポリシーネットワークの更新
    optimizer.zero_grad()
    # ログ確率と割引報酬の要素積を取り、その合計を目的関数として設定
    loss = torch.sum(torch.mul(episode_log_probs, discounted_rewards).mul(-1))
    # 逆伝播とパラメータの更新
    loss.backward()
    optimizer.step()


# 特徴写像
# 離散化
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    ob0,ob1, ob2, ob3 = observation

    digitized = [
        np.digitize(ob0, bins=bins(-4.8, 4.8, NUM_DIZITIZED)),
        np.digitize(ob1, bins=bins(-5, 5, NUM_DIZITIZED)),
        np.digitize(ob2, bins=bins(-0.418, 0.418, NUM_DIZITIZED)),
        np.digitize(ob3, bins=bins(-5, 5, NUM_DIZITIZED))
    ]
    return np.array(digitized)



# パラメータの設定
num_episodes = 30000

# 環境の初期化
env = gym.make('CartPole-v1')
# env = gym.make('InvertedPendulum-v4', render_mode="human") #可視化


# 学習のメインループ
for episode in range(num_episodes):
    state, _ = env.reset()
    state = digitize_state(state)
    # print(state)
    episode_rewards = []
    episode_log_probs = []
    step_num = 0

    while True:
        # 行動の選択
        action, log_prob = select_action(state)
        # if action == 1:
        #     action = 0.001
        # else:
        #     action = -0.001

        #スカラー値のアクションを (1,) の形状のNumPyアレイに変換
        # action = np.array([action])
        # 状態の遷移と報酬の取得
        next_state, reward, done, _, _ = env.step(action)
        # print(next_state)

        # エピソードの報酬と対数確率を記録
        episode_rewards.append(reward)
        episode_log_probs.append(log_prob)

        state = next_state
        state = digitize_state(state)
        # print(state)
        # step_num = step_num + 1

        # エピソードの終了時
        if done:

            # record_episode.append(episode+1)
            # record_step.append(step_num)
            # record_reward.append(sum(episode_rewards))
            # record_episode_log_probs.append(episode_log_probs)
            # ポリシーネットワークの更新
            update_policy(episode_rewards, episode_log_probs)

            # ログの表示
            print("エピソード: {}, 総報酬: {}".format(episode, sum(episode_rewards)))
            break
env.close()

# グラフ表示
# plt.plot(record_episode, record_reward, color="red")
# plt.grid()
# plt.xlabel("episode")
# plt.ylabel("reward")

# # 移動平均を計算する窓サイズを設定
# window_size = 100

# # 報酬の移動平均を計算
# reward_ma = np.convolve(record_reward, np.ones(window_size)/window_size, mode='valid')


# # 報酬をプロット
# plt.plot(record_episode, record_reward, color="red", label="Reward")
# plt.plot(record_episode[window_size-1:], reward_ma, color="orange", label=f"{window_size}-Episode Moving Average")
# plt.grid()
# plt.xlabel("Episode")
# plt.ylabel("Reward")
# plt.legend(loc='upper right')


エピソード: 0, 総報酬: 12.0
エピソード: 1, 総報酬: 15.0
エピソード: 2, 総報酬: 15.0
エピソード: 3, 総報酬: 26.0
エピソード: 4, 総報酬: 10.0
エピソード: 5, 総報酬: 13.0
エピソード: 6, 総報酬: 9.0
エピソード: 7, 総報酬: 12.0
エピソード: 8, 総報酬: 11.0
エピソード: 9, 総報酬: 9.0
エピソード: 10, 総報酬: 11.0
エピソード: 11, 総報酬: 16.0
エピソード: 12, 総報酬: 27.0
エピソード: 13, 総報酬: 8.0
エピソード: 14, 総報酬: 14.0
エピソード: 15, 総報酬: 10.0
エピソード: 16, 総報酬: 20.0
エピソード: 17, 総報酬: 15.0
エピソード: 18, 総報酬: 29.0
エピソード: 19, 総報酬: 10.0
エピソード: 20, 総報酬: 14.0
エピソード: 21, 総報酬: 13.0
エピソード: 22, 総報酬: 15.0
エピソード: 23, 総報酬: 12.0
エピソード: 24, 総報酬: 17.0
エピソード: 25, 総報酬: 9.0
エピソード: 26, 総報酬: 18.0
エピソード: 27, 総報酬: 9.0
エピソード: 28, 総報酬: 10.0
エピソード: 29, 総報酬: 12.0
エピソード: 30, 総報酬: 12.0
エピソード: 31, 総報酬: 14.0
エピソード: 32, 総報酬: 18.0
エピソード: 33, 総報酬: 10.0
エピソード: 34, 総報酬: 12.0
エピソード: 35, 総報酬: 26.0
エピソード: 36, 総報酬: 10.0
エピソード: 37, 総報酬: 10.0
エピソード: 38, 総報酬: 10.0
エピソード: 39, 総報酬: 11.0
エピソード: 40, 総報酬: 16.0
エピソード: 41, 総報酬: 10.0
エピソード: 42, 総報酬: 17.0
エピソード: 43, 総報酬: 13.0
エピソード: 44, 総報酬: 16.0
エピソード: 45, 総報酬: 25.0
エピソード: 46, 総報酬: 12.0
エピソード: 47, 総報酬: 9.0
エピソード: 4